In [6]:
import os
import json
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

def extract_landmark_y(landmarks, name):
    for lm in landmarks:
        if lm["name"] == name and lm["conf"] > 0.5:
            return lm["y"]
    return None

def count_pullup_reps_from_json(json_path, min_prominence=0.01):
    with open(json_path, 'r') as f:
        frames = json.load(f)

    y_series = []
    for frame in frames:
        landmarks = frame.get("landmarks", [])
        y_vals = []

        for key in ["left_shoulder", "right_shoulder", "left_elbow", "right_elbow"]:
            y = extract_landmark_y(landmarks, key)
            if y is not None:
                y_vals.append(y)

        if len(y_vals) >= 3:  # Require at least 3 points to average
            composite_y = np.mean(y_vals)
            y_series.append(composite_y)

    if len(y_series) < 5:
        return 0

    # Smooth and normalize
    y_array = np.array(y_series)
    y_array = gaussian_filter1d(y_array, sigma=2)
    y_array = (y_array - np.min(y_array)) / (np.max(y_array) - np.min(y_array) + 1e-6)

    valleys, _ = find_peaks(-y_array, prominence=min_prominence)
    peaks, _ = find_peaks(y_array, prominence=min_prominence)

    # Count reps as valley → peak → valley sequences
    rep_count = 0
    for i in range(1, len(valleys)):
        prev_valley = valleys[i - 1]
        curr_valley = valleys[i]
        peaks_between = [p for p in peaks if prev_valley < p < curr_valley]
        if peaks_between:
            rep_count += 1

    return rep_count

def process_pullup_json_folder(folder_path, output_csv='pullup_rep_counts_from_json.csv'):
    results = []
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

    for json_file in json_files:
        json_path = os.path.join(folder_path, json_file)
        try:
            reps = count_pullup_reps_from_json(json_path)
            results.append({'Video_Name': json_file, 'Rep_Count': reps})
            print(f"{json_file}: {reps} reps")
        except Exception as e:
            print(f"Error processing {json_file}: {e}")
            results.append({'Video_Name': json_file, 'Rep_Count': 0})

    df = pd.DataFrame(results)
    output_path = os.path.join(folder_path, output_csv)
    df.to_csv(output_path, index=False)
    print(f"\n✅ Results saved to {output_path}")
    return df

# Run the script
if __name__ == "__main__":
    folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\pull Up\Json"
    process_pullup_json_folder(folder)


pull Up_0_pose.json: 7 reps
pull Up_10_pose.json: 0 reps
pull Up_1_pose.json: 3 reps

✅ Results saved to C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\pull Up\Json\pullup_rep_counts_from_json.csv
